In [1]:
from msu_esd import Pipe
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

# Define known constants
K1, K2, K3, K4 = 4.5, 4.5, 4.5, 10  # Loss coefficients

rho = 48.05/32.174  # In slugs per cubic feet
mu = 3.715/(3600*32.174)  # In slugs per (ft s) or lbf*s per ft squared
epsilon = 0.0005  # In ft

D20, D12, D10, D8 = np.array([18.812, 11.938, 10.02, 7.981])/12  # Diameters in ft

Ws = 380  # in ft

p1 = Pipe(D20, 2840, epsilon, rho, mu, K=1.78, C=8)
p2 = Pipe(D10, 2380, epsilon, rho, mu, K=5.53, C=8)
p3 = Pipe(D12, 1300, epsilon, rho, mu, K=1.78, C=8)
p4 = Pipe(D10, 1630, epsilon, rho, mu, K=5.53, C=8)
p5 = Pipe(D12, 3000, epsilon, rho, mu, K=3.28, C=8)
p6 = Pipe(D12, 5000, epsilon, rho, mu, K=3.28, C=8)
p7 = Pipe(D12, 1580, epsilon, rho, mu, K=1.78, C=8)
p8 = Pipe(D8, 1550, epsilon, rho, mu, K=5.53, C=8)
p9 = Pipe(D8, 1550, epsilon, rho, mu, K=7.03, C=8)
p10 = Pipe(D10, 5130, epsilon, rho, mu, K=7.78, C=8)
p11 = Pipe(D12, 1875, epsilon, rho, mu, K=2.53, C=8)

def unbalanced(x):
    Q1, Q2, Q3, Q4, Q5, Q6, Q7, Q8, Q9, Q10, Q11 = x
    # all expressions need to be set to zero
    return [
        Q1 - Q2 - Q3,
        Q6 - Q2 - Q4,
        Q3 - Q4 - Q5,
        Q5 - Q10 - Q7,
        Q7 - Q8 - Q9,
        Q11 - Q6 - Q7,
        K1*Q2*abs(Q2) + p2.h(Q2) + p6.h(Q6) + p11.h(Q11) + p1.h(Q1) - Ws + 0.1*Q1*abs(Q1),
        p4.h(Q4) + K3*Q4*abs(Q4) - p2.h(Q2) - K1*Q2*abs(Q2) + p3.h(Q3),
        p7.h(Q7) + p8.h(Q8) + K4*Q8*abs(Q8) - p6.h(Q6) - p4.h(Q4) - K3*Q4*abs(Q4) + p5.h(Q5),
        p9.h(Q9) + K4*Q9*abs(Q9) - p8.h(Q8) - K4*Q8*abs(Q8),
        p10.h(Q10) + K2*Q10*abs(Q10) - p11.h(Q11) - p9.h(Q9) - K4*Q9*abs(Q9) - p7.h(Q7)
    ]

def test(pipes_, flow_rates):
    lines = range(1, len(pipes_) + 1)
    Q_values, V_values = [], []
    AHU_1_3, AHU_4 = [], []
    V_less = []

    for pipe_, Q_, i in zip(pipes_, flow_rates, lines):
        Q_values.append(Q_)
        V_ = pipe_.V(Q_)
        V_values.append(V_)

        if V_ < 9:
            V_less.append('Yes')
        else:
            V_less.append('No')

        if i in [2, 4, 10]:
            if 2.713 <= Q_ < 2.808:
                AHU_1_3.append('Yes')
            else:
                AHU_1_3.append('No')
        else:
            AHU_1_3.append('-')

        if i in [8, 9]:
            if 1.272 <= Q_ < 1.316:
                AHU_4.append('Yes')
            else:
                AHU_4.append('No')
        else:
            AHU_4.append('-')

    return pd.DataFrame({'Pipe': lines, 'Q': Q_values, 'V': V_values, 'V<9': V_less, '2.713<=Q<2.808': AHU_1_3, r'1.272<=Q<1.316': AHU_4})

Q_guess = np.array([5, 1, 4, 2, 2, 3, 1, 3, -2, 1, 4])  # This does satisfy mass conservation
unbalanced_sol = fsolve(unbalanced, Q_guess)
pipes = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11]
iteration_test = test(pipes, unbalanced_sol)
iteration_test

,Pipe,Q,V,V<9,2.713<=Q<2.808,1.272<=Q<1.316
0,1,14.525994,7.525728,Yes,-,-
1,2,4.005309,7.314307,Yes,No,-
2,3,10.520685,13.534851,No,-,-
3,4,2.748716,5.019577,Yes,Yes,-
4,5,7.771968,9.998630,No,-,-
5,6,6.754025,8.689047,Yes,-,-
6,7,3.779888,4.862822,Yes,-,-
7,8,1.895431,5.455894,Yes,-,No
8,9,1.884457,5.424308,Yes,-,No
9,10,3.992080,7.290150,Yes,No,-
